In [1]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error
 
#X = 
#y = 
 
# Set up parameters
time_steps = 20
hidden_units = 2
epochs = 30
 
# Create a traditional RNN network
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model
 
model_RNN = create_RNN(hidden_units=hidden_units, dense_units=1, input_shape=(time_steps,1), 
                   activation=['tanh', 'tanh'])
 
# Generate the dataset for the network
#trainX, trainY, testX, testY, scaler  = 
model_RNN.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)
 
 
# Evalute model
train_mse = model_RNN.evaluate(trainX, trainY)
test_mse = model_RNN.evaluate(testX, testY)
 
# Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)
 
 
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
    
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')    
    return model    
 
# Create the model with attention, train and evaluate
model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1, 
                                  input_shape=(time_steps,1), activation='tanh')
model_attention.summary()    
 
 
model_attention.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)
 
# Evalute model
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)
 
# Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

Epoch 1/30
826/826 - 5s - loss: 0.0029
Epoch 2/30
826/826 - 4s - loss: 0.0028
Epoch 3/30
826/826 - 4s - loss: 0.0026
Epoch 4/30
826/826 - 4s - loss: 0.0025
Epoch 5/30
826/826 - 4s - loss: 0.0023
Epoch 6/30
826/826 - 4s - loss: 0.0022
Epoch 7/30
826/826 - 4s - loss: 0.0020
Epoch 8/30
826/826 - 4s - loss: 0.0019
Epoch 9/30
826/826 - 4s - loss: 0.0017
Epoch 10/30
826/826 - 4s - loss: 0.0016
Epoch 11/30
826/826 - 4s - loss: 0.0015
Epoch 12/30
826/826 - 4s - loss: 0.0014
Epoch 13/30
826/826 - 4s - loss: 0.0012
Epoch 14/30
826/826 - 4s - loss: 0.0011
Epoch 15/30
826/826 - 4s - loss: 9.9040e-04
Epoch 16/30
826/826 - 4s - loss: 8.8794e-04
Epoch 17/30
826/826 - 4s - loss: 7.9339e-04
Epoch 18/30
826/826 - 4s - loss: 7.0237e-04
Epoch 19/30
826/826 - 4s - loss: 6.0552e-04
Epoch 20/30
826/826 - 4s - loss: 5.2039e-04
Epoch 21/30
826/826 - 4s - loss: 4.4582e-04
Epoch 22/30
826/826 - 3s - loss: 3.7892e-04
Epoch 23/30
826/826 - 4s - loss: 3.2269e-04
Epoch 24/30
826/826 - 4s - loss: 2.6848e-04
Epoch 25/

In [1]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
 

 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini
 
# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root
 
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']
 
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample
 
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)
 
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)
 
# Test the random forest algorithm
seed(2)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

NameError: name 'load_csv' is not defined

In [1]:

# Load Python libraries
from sklearn import metrics, ensemble
import xgboost as xgb
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings


import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# create dummy data
X,y = make_classification(n_samples=1000, n_features=10, n_informative=3, random_state=1)

# fit NN
model = Sequential()
model.add(Dense(units=100,input_dim=10,activation='relu'))
model.add(Dense(units=40,activation='relu'))
model.add(Dense(units=10,input_dim=10,activation='relu'))
model.add(Dense(units=4,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X,y, epochs=10)

# extract feature from NN
extraction_model = Model(model.input, model.layers[1].output)
# in model.layers you can find all the list of available layers' output
# we select the output of our second Dense layer
# the second Dense return 2D data
new_X = extraction_model.predict(X) # (n_sample, hidden_dim)

# fit a tabular model
rf = CatBoostClassifier()
rf.fit(new_X, y)
print(rf.score(new_X, y))

Epoch 1/10
32/32 [==============================] - 1s 1ms/step - loss: 0.5684 - accuracy: 0.6410
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.3974 - accuracy: 0.9120
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.2460 - accuracy: 0.9400
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1768 - accuracy: 0.9500
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1482 - accuracy: 0.9530
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1374 - accuracy: 0.9590
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1294 - accuracy: 0.9610
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1226 - accuracy: 0.9630
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1162 - accuracy: 0.9680
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.1113 - accuracy: 0.9660
Learning rate set t

144:	learn: 0.1070807	total: 895ms	remaining: 5.28s
145:	learn: 0.1066341	total: 900ms	remaining: 5.26s
146:	learn: 0.1057895	total: 905ms	remaining: 5.25s
147:	learn: 0.1051478	total: 910ms	remaining: 5.24s
148:	learn: 0.1045493	total: 916ms	remaining: 5.23s
149:	learn: 0.1038788	total: 920ms	remaining: 5.21s
150:	learn: 0.1032336	total: 925ms	remaining: 5.2s
151:	learn: 0.1026834	total: 929ms	remaining: 5.18s
152:	learn: 0.1021158	total: 933ms	remaining: 5.17s
153:	learn: 0.1015498	total: 940ms	remaining: 5.16s
154:	learn: 0.1010965	total: 945ms	remaining: 5.15s
155:	learn: 0.1007033	total: 950ms	remaining: 5.14s
156:	learn: 0.1002094	total: 954ms	remaining: 5.12s
157:	learn: 0.0998047	total: 959ms	remaining: 5.11s
158:	learn: 0.0994286	total: 964ms	remaining: 5.1s
159:	learn: 0.0990851	total: 969ms	remaining: 5.09s
160:	learn: 0.0986361	total: 973ms	remaining: 5.07s
161:	learn: 0.0981537	total: 978ms	remaining: 5.06s
162:	learn: 0.0975294	total: 984ms	remaining: 5.05s
163:	learn: 0.

320:	learn: 0.0592231	total: 1.78s	remaining: 3.76s
321:	learn: 0.0590429	total: 1.78s	remaining: 3.75s
322:	learn: 0.0588839	total: 1.79s	remaining: 3.75s
323:	learn: 0.0587386	total: 1.79s	remaining: 3.74s
324:	learn: 0.0585574	total: 1.8s	remaining: 3.73s
325:	learn: 0.0584120	total: 1.8s	remaining: 3.73s
326:	learn: 0.0583650	total: 1.81s	remaining: 3.72s
327:	learn: 0.0582674	total: 1.81s	remaining: 3.72s
328:	learn: 0.0581365	total: 1.82s	remaining: 3.71s
329:	learn: 0.0580005	total: 1.82s	remaining: 3.7s
330:	learn: 0.0579003	total: 1.83s	remaining: 3.7s
331:	learn: 0.0578477	total: 1.83s	remaining: 3.69s
332:	learn: 0.0577721	total: 1.84s	remaining: 3.68s
333:	learn: 0.0576787	total: 1.84s	remaining: 3.68s
334:	learn: 0.0575237	total: 1.85s	remaining: 3.67s
335:	learn: 0.0574419	total: 1.85s	remaining: 3.66s
336:	learn: 0.0573660	total: 1.86s	remaining: 3.66s
337:	learn: 0.0572654	total: 1.86s	remaining: 3.65s
338:	learn: 0.0571354	total: 1.87s	remaining: 3.64s
339:	learn: 0.05

494:	learn: 0.0420473	total: 2.67s	remaining: 2.72s
495:	learn: 0.0419934	total: 2.67s	remaining: 2.71s
496:	learn: 0.0419844	total: 2.68s	remaining: 2.71s
497:	learn: 0.0419243	total: 2.68s	remaining: 2.7s
498:	learn: 0.0418448	total: 2.69s	remaining: 2.7s
499:	learn: 0.0417898	total: 2.69s	remaining: 2.69s
500:	learn: 0.0417298	total: 2.7s	remaining: 2.69s
501:	learn: 0.0416410	total: 2.7s	remaining: 2.68s
502:	learn: 0.0415855	total: 2.71s	remaining: 2.67s
503:	learn: 0.0415210	total: 2.71s	remaining: 2.67s
504:	learn: 0.0414118	total: 2.72s	remaining: 2.66s
505:	learn: 0.0413511	total: 2.72s	remaining: 2.66s
506:	learn: 0.0412639	total: 2.73s	remaining: 2.65s
507:	learn: 0.0411568	total: 2.73s	remaining: 2.65s
508:	learn: 0.0410846	total: 2.74s	remaining: 2.64s
509:	learn: 0.0410444	total: 2.74s	remaining: 2.64s
510:	learn: 0.0409827	total: 2.75s	remaining: 2.63s
511:	learn: 0.0409395	total: 2.75s	remaining: 2.62s
512:	learn: 0.0408487	total: 2.76s	remaining: 2.62s
513:	learn: 0.04

667:	learn: 0.0318192	total: 3.55s	remaining: 1.76s
668:	learn: 0.0317717	total: 3.55s	remaining: 1.76s
669:	learn: 0.0317336	total: 3.56s	remaining: 1.75s
670:	learn: 0.0316746	total: 3.56s	remaining: 1.75s
671:	learn: 0.0316225	total: 3.57s	remaining: 1.74s
672:	learn: 0.0315867	total: 3.58s	remaining: 1.74s
673:	learn: 0.0315075	total: 3.58s	remaining: 1.73s
674:	learn: 0.0314584	total: 3.59s	remaining: 1.73s
675:	learn: 0.0314235	total: 3.59s	remaining: 1.72s
676:	learn: 0.0313826	total: 3.6s	remaining: 1.72s
677:	learn: 0.0313316	total: 3.6s	remaining: 1.71s
678:	learn: 0.0312935	total: 3.61s	remaining: 1.71s
679:	learn: 0.0312623	total: 3.61s	remaining: 1.7s
680:	learn: 0.0311560	total: 3.62s	remaining: 1.69s
681:	learn: 0.0311145	total: 3.62s	remaining: 1.69s
682:	learn: 0.0310378	total: 3.63s	remaining: 1.68s
683:	learn: 0.0310202	total: 3.63s	remaining: 1.68s
684:	learn: 0.0309906	total: 3.64s	remaining: 1.67s
685:	learn: 0.0309652	total: 3.64s	remaining: 1.67s
686:	learn: 0.0

841:	learn: 0.0253330	total: 4.43s	remaining: 832ms
842:	learn: 0.0253166	total: 4.44s	remaining: 827ms
843:	learn: 0.0252866	total: 4.45s	remaining: 822ms
844:	learn: 0.0252772	total: 4.45s	remaining: 817ms
845:	learn: 0.0252601	total: 4.46s	remaining: 811ms
846:	learn: 0.0252462	total: 4.46s	remaining: 806ms
847:	learn: 0.0251678	total: 4.47s	remaining: 801ms
848:	learn: 0.0251511	total: 4.47s	remaining: 796ms
849:	learn: 0.0251047	total: 4.48s	remaining: 791ms
850:	learn: 0.0250908	total: 4.48s	remaining: 785ms
851:	learn: 0.0250420	total: 4.49s	remaining: 780ms
852:	learn: 0.0250179	total: 4.5s	remaining: 775ms
853:	learn: 0.0250021	total: 4.5s	remaining: 769ms
854:	learn: 0.0249932	total: 4.5s	remaining: 764ms
855:	learn: 0.0249778	total: 4.51s	remaining: 759ms
856:	learn: 0.0249688	total: 4.52s	remaining: 754ms
857:	learn: 0.0249290	total: 4.52s	remaining: 749ms
858:	learn: 0.0249032	total: 4.53s	remaining: 743ms
859:	learn: 0.0248876	total: 4.53s	remaining: 738ms
860:	learn: 0.0